In [ ]:
#export .xls from SEMrush for a period / day, save to desktop
#https://www.semrush.com/tracking/landscape/3317965.html?domain_1=expert.nl&domain_2=coolblue.nl&domain_3=wehkamp.nl&domain_4=mediamarkt.nl&domain_5=bcc.nl

In [ ]:
#import all libraries, widgets and ignore warnings
import win32com.client as win32
import os
import pandas as pd
import numpy as np
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interactive, widgets, interact, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML
from pivottablejs import pivot_ui

import warnings
warnings.simplefilter('ignore')

In [ ]:
#convert .xls to .xlsx
fname = "C:\\Users\\{USER_NAME}\\{DOWNLOAD_LOCATION}\\{exportSEMrush.xls}"
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fname)
#fileformet 51 == xlsx format
wb.SaveAs("C:\\Users\\{USER_NAME}}\\Desktop\\PythonProjects\\reporting\\rankings_converted\\20200930", 
          FileFormat = 51)
wb.Close()
excel.Application.Quit()

In [ ]:
#read .xlsx into dataframe
df_20200930 = pd.read_excel("./rankings_converted/20200930.xlsx", header=7)
#replace empty landing pages
df_20200930.replace(r'\s+',np.nan,regex=True).replace('',np.nan, inplace=True)
#add date to columns, date should be changed manually
df_20200930['Datum'] = '2020-9-30'
#to avoid a warning
pd.options.mode.chained_assignment = None 
#date to datetime64 dtype
df_20200930['Datum'] =  pd.to_datetime(df_20200930['Datum'], format='%Y%m%d', infer_datetime_format=True)
#rename columns
df_20200930.rename(columns={'tags':'Tags','*.expert.nl/*_20200930': 'Rankings_Expert', '*.expert.nl/*_20200930_landing': 'Expert_url','*.bcc.nl/*_20200930': 'Rankings_BCC', '*.bcc.nl/*_20200930_landing': 'BCC_url','*.coolblue.nl/*_20200930': 'Rankings_Coolblue', '*.coolblue.nl/*_20200930_landing': 'Coolblue_url','*.mediamarkt.nl/*_20200930': 'Rankings_Mediamarkt', '*.mediamarkt.nl/*_20200930_landing': 'Mediamarkt_url','*.wehkamp.nl/*_20200930': 'Rankings_Wehkamp', '*.wehkamp.nl/*_20200930_landing': 'Wehkamp_url','Search Volume': 'Search_Volume'}, inplace=True)
#define which columns will be used
use_cols_20200930 = df_20200930[['Datum','Keyword','Tags','Rankings_Expert','Expert_url','Rankings_BCC','BCC_url','Rankings_Coolblue','Coolblue_url','Rankings_Mediamarkt','Mediamarkt_url','Rankings_Wehkamp','Wehkamp_url','CPC','Search_Volume']]
#check if it works
df_20200930.head()

In [ ]:
#save to new .xlsx file
use_cols_20200930.to_excel('./use_cols/20200930.xlsx')

In [ ]:
#combine all files from directory
files = [file for file in os.listdir('./use_cols/')]
workfile = pd.DataFrame()
for file in files:
    df = pd.read_excel('./use_cols/'+file)
    workfile = pd.concat([workfile, df])
#check results
workfile.info()

In [ ]:
#categorize all keywords from txt file
tags = pd.read_csv('./workfile/tags.txt', sep=';')
#merge tags on keywords, check and export to new .csv (workfile)
output = workfile.merge(tags, on='Tags')
output.head()
output.to_csv('./workfile/output.csv', index=False)

# CREATE VISIBILITY REPORT

In [ ]:
#open workfile
df = pd.read_csv('./workfile/output.csv',) 
df.drop('Unnamed: 0', axis=1, inplace=True) #df.drop('Unnamed: 0.1', axis=1, inplace=True) <<-- somehow this gives errors...
#replace empty landing pages
df.replace(r'\s+',np.nan,regex=True).replace('',np.nan, inplace=True)

In [ ]:
#create visibility_dataframe for each website
#VIS_EXPERT
df_vis_expert = pd.read_excel('./visibility_competitors/vis_expert.xlsx')
df_vis_calc = df.merge(df_vis_expert,on='Rankings_Expert', how='outer')
df_vis_calc['Visibility_Expert'] = df_vis_calc.Visibility_Expert.astype(float)

#VIS_BCC
df_vis_bcc = pd.read_excel('./visibility_competitors/vis_bcc.xlsx')
df_vis_calc2 = df_vis_calc.merge(df_vis_bcc,on='Rankings_BCC', how='outer')
df_vis_calc2['Visibility_BCC'] = df_vis_calc2.Visibility_BCC.astype(float)

#VIS_Coolblue
df_vis_coolblue = pd.read_excel('./visibility_competitors/vis_coolblue.xlsx')
df_vis_calc3 = df_vis_calc2.merge(df_vis_coolblue,on='Rankings_Coolblue', how='outer')
df_vis_calc3['Visibility_Coolblue'] = df_vis_calc3.Visibility_Coolblue.astype(float)

#VIS_Mediamarkt
df_vis_mediamarkt = pd.read_excel('./visibility_competitors/vis_mediamarkt.xlsx')
df_vis_calc4 = df_vis_calc3.merge(df_vis_mediamarkt,on='Rankings_Mediamarkt', how='outer')
df_vis_calc4['Visibility_Mediamarkt'] = df_vis_calc4.Visibility_Mediamarkt.astype(float)

#VIS_Wehkamp
df_vis_wehkamp = pd.read_excel('./visibility_competitors/vis_wehkamp.xlsx')
df_vis_calc5 = df_vis_calc4.merge(df_vis_wehkamp,on='Rankings_Wehkamp', how='outer')
df_vis_calc5['Visibility_Wehkamp'] = df_vis_calc5.Visibility_Wehkamp.astype(float)

In [ ]:
#rename file and do some editing
df_workfile = df_vis_calc5.dropna(subset=['Keyword'])
df_workfile.rename(columns={'Tag2':'Categorie'}, inplace=True)
#reset index
df_workfile.reset_index()
#check if it works
df_workfile.head()

In [ ]:
#create visibilty_dataframe for all websites and export to .csv
visibility = df_workfile[['Datum','Keyword','Tags','Categorie','Visibility_Expert','Visibility_Coolblue','Visibility_Mediamarkt','Visibility_Wehkamp','Visibility_BCC']]
visibility.fillna(0, inplace=True)
visibility.to_csv('./workfile/visibility.csv', index=False)

In [ ]:
#we are gonna use the following export later on:
ranking = df_workfile[['Datum','Keyword','Search_Volume','Tags','Categorie','Rankings_Expert','Rankings_Coolblue','Rankings_Mediamarkt','Rankings_Wehkamp','Rankings_BCC']]
ranking.fillna(0, inplace=True)
ranking.to_csv('./workfile/ranking.csv', index=False)

In [ ]:
#new dataframe
dataset = pd.read_csv('./workfile/visibility.csv')
#with some edits
dataset.fillna(0, inplace=True)
# vis.drop('Unnamed: 0', axis=1, inplace=True)
dataset.groupby(['Categorie']).mean().reset_index()
dataset['Datum'] =  pd.to_datetime(visibility['Datum'], format='%Y%m%d', infer_datetime_format=True)
dataset.info()

# NOW THE FUN PART

In [ ]:
#plot is like it's hot
pivot = dataset.groupby(['Datum']).mean()
pivot.plot(kind='line',title='Visibility Overtime')
plt.xlabel('Period', fontsize=14)
plt.ylabel('Search Visibility in Google', fontsize=14)
plt.style.use('Solarize_Light2')
plt.show()

# NOW EVEN MORE FUN! INTERACTIVE PLOTTING ^_^

In [ ]:
#create a widget
tags = widgets.Dropdown(
    options=['All'] + list(dataset['Categorie'].unique()),
    value='All',
    description='Category:',
        
)

def plotit(tags):
    if 0 > 1:
        print('Nothing to show')  
    else: 
        dataset2 = dataset.copy()
        if tags != 'All':
            dataset2 = dataset2[dataset2.Categorie == tags]
            plt.style.use('Solarize_Light2')
            plt.show()
        
        if len(dataset2) > 0:
            dataset2.groupby(['Datum']).mean().plot(kind='line',figsize=(14,4))
            plt.xlabel('Period', fontsize=14)
            plt.ylabel('Search Visibility', fontsize=14)
            plt.title('Search Visibility in Google for {}'.format(tags), fontsize=20)
            plt.style.use('Solarize_Light2')
            plt.show()
        else: 
            print('Nothing to show')
            
#get a 100% width view for the full experience on this one (optional)
display(HTML("<style>.container { width:100% !important; }</style>"))

interactive(plotit, tags=tags)

# CREATE RANKING REPORT

In [ ]:
#get the data in place
rnk = pd.read_csv('./workfile/ranking.csv') # <<-- remember we created this file earlier? ^_^
rnk.fillna(0, inplace=True)
rnk['Datum'] =  pd.to_datetime(rnk['Datum'], format='%Y%m%d', infer_datetime_format=True)
rnk[rnk.eq(0)] = np.nan
rnk.info()

In [ ]:
#create a widget
tags = widgets.Dropdown(
    options=['All'] + list(rnk['Categorie'].unique()),
    value='All',
    description='Category:',        
)

def plotit(tags):
    if 0 > 1:
        print('Nothing to show')  
    else: 
        dataset2 = rnk.copy()
        if tags != 'All':
            dataset2 = dataset2[dataset2.Categorie == tags]
            plt.style.use('Solarize_Light2')
            plt.show()
        
        if len(dataset2) > 0:
            dataset2.drop('Search_Volume', axis=1, inplace=True)
            dataset2.groupby(['Datum']).mean().plot(kind='line',figsize=(14,4))
            plt.xlabel('Period', fontsize=14)
            plt.ylabel('Rankings in Google', fontsize=14)
            plt.gca().invert_yaxis()
            plt.title('Rankings in Google for category {}'.format(tags), fontsize=20)
            plt.style.use('Solarize_Light2')
            plt.show()
        else: 
            print('Nothing to show')

#get a 100% width view for the full experience on this one (optional)
display(HTML("<style>.container { width:100% !important; }</style>"))

interactive(plotit, tags=tags)

In [ ]:
## WE CAN DO THE SAME FOR ALL EACH KEYWORD ##
kws = widgets.Dropdown(
    options=['All'] + list(rnk['Keyword'].unique()),
    value='All',
    description='Keyword:',        
)

def plotit2(kws):
    if 0 > 1:
        print('Nothing to show')  
    else: 
        dataset2 = rnk.copy()
        if kws != 'All':
            dataset2 = dataset2[dataset2.Keyword == kws]
            plt.style.use('Solarize_Light2')
            plt.show()
        
        if len(dataset2) > 0:
            dataset2.drop('Search_Volume', axis=1, inplace=True)
            dataset2.groupby(['Datum']).mean().plot(kind='line',figsize=(14,4))
            plt.xlabel('Period', fontsize=14)
            plt.ylabel('Rankings in Google', fontsize=14)
            plt.gca().invert_yaxis()
            plt.title('Rankings in Google for keyword [{}]'.format(kws), fontsize=20)
            plt.style.use('Solarize_Light2')
            plt.show()
        else: 
            print('Nothing to show') 
            
#get a 100% width view for the full experience on this one (optiional)
display(HTML("<style>.container { width:100% !important; }</style>"))

interactive(plotit, tags=tags)

# PIVOT INTO ACTIONABLE AND INTERACTIVE TABLE

In [ ]:
#do some data tweaks
pivot_df = pd.read_csv('./workfile/ranking.csv')
pivot_df[pivot_df.eq(0)] = np.nan
pivot_df["Search_Volume"] = pivot_df["Search_Volume"].astype(int)
pivot_df['Datum'] =  pd.to_datetime(pivot_df['Datum'], format='%Y%m%d', infer_datetime_format=True)
pivot_df.info()

In [ ]:
rankings = pivot_df[['Datum', 'Keyword', 'Categorie', 'Search_Volume', 'Tags', 'Rankings_BCC']]
rankings.sort_values(by='Datum')

In [ ]:
#create interactive pivot (to see in new tab: go to http://localhost:{PORT}/files/{YOUR_PROJECT_LOCATION}/pivottablejs.html)
pivot_ui(rankings,      
    cols= ["Datum"],
    rows= ["Search_Volume"],
    rowOrder= "value_a_to_z",
    rendererOptions= {
        "c3": { "data": {"colors": {
            "Liberal": '#dc3912', "Conservative": '#3366cc', "NDP": '#ff9900',
            "Green":'#109618', 'Bloc Quebecois': '#990099'
        }}}
    }
)